In [1]:
#Standard Libraries
import numpy as np
import pandas as pd
import os
import time

#Data Preprocessing
from sklearn.model_selection import train_test_split

#Feature Selectors
from sklearn.feature_selection import chi2

#Models
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression

#Model Evaluation
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.metrics import mean_squared_error, r2_score

import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('daily.csv')
data.head(10)

,date,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,hospitalized,total,totalTestResults,posNeg,fips,deathIncrease,hospitalizedIncrease,negativeIncrease,positiveIncrease,totalTestResultsIncrease
0,20200422,AK,335.0,11824.0,NaN,39.0,36.0,NaN,NaN,NaN,...,36.0,12159.0,12159.0,12159.0,2,0.0,0.0,1034.0,6.0,1040.0
1,20200422,AL,5465.0,43295.0,NaN,NaN,730.0,NaN,288.0,NaN,...,730.0,48760.0,48760.0,48760.0,1,17.0,31.0,0.0,234.0,234.0
2,20200422,AR,2276.0,27437.0,NaN,97.0,291.0,NaN,NaN,23.0,...,291.0,29713.0,29713.0,29713.0,5,-1.0,0.0,2223.0,49.0,2272.0
3,20200422,AS,0.0,3.0,17.0,NaN,NaN,NaN,NaN,NaN,...,NaN,20.0,3.0,3.0,60,0.0,0.0,0.0,0.0,0.0
4,20200422,AZ,5459.0,51142.0,NaN,664.0,NaN,300.0,NaN,195.0,...,NaN,56601.0,56601.0,56601.0,4,21.0,0.0,1241.0,208.0,1449.0
5,20200422,CA,35396.0,429931.0,NaN,4984.0,NaN,1551.0,NaN,NaN,...,NaN,465327.0,465327.0,465327.0,6,86.0,0.0,163092.0,2135.0,165227.0
6,20200422,CO,10447.0,38257.0,NaN,851.0,2003.0,NaN,NaN,NaN,...,2003.0,48704.0,48704.0,48704.0,8,37.0,123.0,897.0,341.0,1238.0
7,20200422,CT,22469.0,47449.0,NaN,1972.0,NaN,NaN,NaN,NaN,...,NaN,69918.0,69918.0,69918.0,9,121.0,0.0,3617.0,2109.0,5726.0
8,20200422,DC,3206.0,12296.0,NaN,402.0,NaN,120.0,NaN,69.0,...,NaN,15502.0,15502.0,15502.0,11,15.0,0.0,455.0,108.0,563.0
9,20200422,DE,3200.0,13353.0,NaN,269.0,NaN,NaN,NaN,NaN,...,NaN,16553.0,16553.0,16553.0,10,7.0,0.0,-372.0,269.0,-103.0


In [3]:
X = data[['positive', 'negative', "hospitalizedCurrently", "inIcuCurrently", 'onVentilatorCurrently', 'death', 'hospitalized', 'posNeg', 'totalTestResults']]
X.head(5)

,positive,negative,hospitalizedCurrently,inIcuCurrently,onVentilatorCurrently,death,hospitalized,posNeg,totalTestResults
0,335.0,11824.0,39.0,NaN,NaN,9.0,36.0,12159.0,12159.0
1,5465.0,43295.0,NaN,NaN,NaN,194.0,730.0,48760.0,48760.0
2,2276.0,27437.0,97.0,NaN,23.0,42.0,291.0,29713.0,29713.0
3,0.0,3.0,NaN,NaN,NaN,NaN,NaN,3.0,3.0
4,5459.0,51142.0,664.0,300.0,195.0,229.0,NaN,56601.0,56601.0


In [4]:
X = X.fillna(0)

In [5]:
X.head(5)

,positive,negative,hospitalizedCurrently,inIcuCurrently,onVentilatorCurrently,death,hospitalized,posNeg,totalTestResults
0,335.0,11824.0,39.0,0.0,0.0,9.0,36.0,12159.0,12159.0
1,5465.0,43295.0,0.0,0.0,0.0,194.0,730.0,48760.0,48760.0
2,2276.0,27437.0,97.0,0.0,23.0,42.0,291.0,29713.0,29713.0
3,0.0,3.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0
4,5459.0,51142.0,664.0,300.0,195.0,229.0,0.0,56601.0,56601.0


In [6]:
y = data[['recovered']]
y.head()

,recovered
0,196.0
1,NaN
2,863.0
3,NaN
4,1265.0


In [7]:
y = y.fillna(0)

In [8]:
y.head()

,recovered
0,196.0
1,0.0
2,863.0
3,0.0
4,1265.0


In [9]:
y.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2676 entries, 0 to 2675
Data columns (total 1 columns):
recovered    2676 non-null float64
dtypes: float64(1)
memory usage: 21.0 KB


In [10]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2676 entries, 0 to 2675
Data columns (total 9 columns):
positive                 2676 non-null float64
negative                 2676 non-null float64
hospitalizedCurrently    2676 non-null float64
inIcuCurrently           2676 non-null float64
onVentilatorCurrently    2676 non-null float64
death                    2676 non-null float64
hospitalized             2676 non-null float64
posNeg                   2676 non-null float64
totalTestResults         2676 non-null float64
dtypes: float64(9)
memory usage: 188.3 KB


In [11]:
features_train, features_test, target_train, target_test = train_test_split(X, y, test_size=0.1, random_state=37)

In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [14]:
#Logistic and Binary Classifier l2
classifier = LogisticRegression(penalty='l2')
model = classifier.fit(features_train, target_train)
test_predictions = model.predict(features_test)
train_predictions = model.predict(features_train)
accuracy_test = accuracy_score(target_test, test_predictions)
accuracy_train = accuracy_score(target_train , train_predictions)

In [15]:
print (accuracy_test)

0.7052238805970149


In [16]:
print(accuracy_train)

0.6980897009966778


In [17]:
#Naive Bayes Category Binary Classifier
classifier = BernoulliNB()
model = classifier.fit(features_train, target_train)
test_predictions = model.predict(features_test)
train_predictions = model.predict(features_train)
accuracy_test = accuracy_score(target_test, test_predictions)
accuracy_train = accuracy_score(target_train , train_predictions)


In [18]:
print (accuracy_test)

0.7014925373134329


In [19]:
print(accuracy_train)

0.7196843853820598


In [20]:
regr = LinearRegression().fit(X, y)

In [22]:
regr.score(X, y)

0.9283930952390466

In [23]:
regr.coef_

array([[-0.04955701,  0.03430863,  0.37394962, -0.25024379, -0.20579152,
         1.09162363,  0.41877947, -0.01524838, -0.01524838]])

In [24]:
regr.intercept_

array([32.47747954])

In [25]:
# Make predictions using the testing set
y_pred = regr.predict(features_test)

In [26]:
# Train the model using the training sets
regr.fit(features_train, target_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [27]:
# The coefficients
print('Coefficients: \n', regr.coef_)
# The mean squared error
print('Mean squared error: %.2f'
      % mean_squared_error(target_test, y_pred))
# The coefficient of determination: 1 is perfect prediction
print('Coefficient of determination: %.2f'
      % r2_score(target_test, y_pred))

Coefficients: 
 [[-0.04606215  0.03206044  0.33245163 -0.22031694 -0.05796367  0.96013684
   0.43221171 -0.01400171 -0.01400171]]
Mean squared error: 303176.56
Coefficient of determination: 0.87
